# Convolutional Neural Networks

Sources:

* http://deeplearning.net/tutorial/lenet.html
* http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/
* http://colah.github.io/posts/2014-07-Conv-Nets-Modular/
* http://colah.github.io/posts/2014-07-Understanding-Convolutions/

## Introduction

In the most basic sense, a convolutional neural network is a kind of neural network that uses many copies of the same neuron. This allows the network express computationally large models while still keeping the total number of parameters that must be learned small. They are biologically inspired from [work on the visual cortex](http://deeplearning.net/tutorial/references.html#hubel68). The visual corext contains a complex arrangement of cells that are sensitive to subregions of the full field. These cells act as local filters and come in two basic varieties: simple cells respond to edge-like patterns and larger, more complex cells with larger receptive fields operate in a locally invariant fashion with respect to the position of the pattern in question.

Many different visually inspired models can be found in the literature, e.g.: NeoCgnitron [Fukushima](http://deeplearning.net/tutorial/references.html#fukushima), [HMAX](http://deeplearning.net/tutorial/references.html#serre07), and LeNet-5 [LeCun](http://deeplearning.net/tutorial/references.html#lecun98). We are experimenting with a version of LeCun's model for MINERvA data.

## Performance / Motivation

One of the great recent breakthroughs in computer vision and image classification came from [Krizhevsky, Sutskever, and Hinton in 2012](http://www.cs.toronto.edu/~fritz/absps/imagenet.pdf). The authors used GPUs to train a very large, deep neural network and used a number of innovations (a new kind of neuron, a [rectified linear unit](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)), and "drop out" to reduce overfitting). 

Their architecture was very complex, with five convolutional layers, with interspersed pooling (more on this shortly), and three fully-connected multilayer perceptron neural networks. The layers were split at the very beginning over two GPUs, with no information flow between the two halves.

<img src="http://colah.github.io/posts/2014-07-Conv-Nets-Modular/img/KSH-arch.png">

(Img src: http://colah.github.io/posts/2014-07-Conv-Nets-Modular/)

Their network was trained to classify images into a thousand different categories. So we would expect random guesses to be good about 0.1% of the time, but their network was good 63% of the time, and one of the best categories was right 85% of the time.

<img src="http://colah.github.io/posts/2014-07-Conv-Nets-Modular/img/KSH-results.png">

(Img src: http://colah.github.io/posts/2014-07-Conv-Nets-Modular/, [Krizhevsky et al 2012](http://www.cs.toronto.edu/~fritz/absps/imagenet.pdf))

Examining the behavior of the two halves of the network was very illuminating. The two sides always specialized. Despite random initialization, the two sides ended up focusing on very different things:

<img src="http://colah.github.io/posts/2014-07-Conv-Nets-Modular/img/KSH-filters.png">

(Img src: http://colah.github.io/posts/2014-07-Conv-Nets-Modular/, [Krizhevsky et al 2012](http://www.cs.toronto.edu/~fritz/absps/imagenet.pdf))

The authors did not choose certain filters to be edge detectors or focus on color or texture - the network learned these features automatically.

## What is Convolution?

Convolution is best thought of as a kernel/filter/feature-detector that one slides over the image/data.

<img src="http://deeplearning.stanford.edu/wiki/images/6/6c/Convolution_schematic.gif">

(GIF courtesy of: http://deeplearning.stanford.edu/wiki/images/6/6c/Convolution_schematic.gif)

In this example, the fiter is a $3 \times 3$ matrix that we multiply element-wise with the original image, and then sum. Different convolutional filters do different things to an image.

Averaging each pixel with its neighbor blurs an image.

\begin{equation}
\begin{bmatrix} 
0 & 0 & 0 & 0 & 0 \\  
0 & 1 & 1 & 1 & 0 \\  
0 & 1 & 1 & 1 & 0 \\  
0 & 1 & 1 & 1 & 0 \\  
0 & 0 & 0 & 0 & 0 \\  
\end{bmatrix}
\end{equation}

<img src="http://docs.gimp.org/en/images/filters/examples/generic-taj-convmatrix-blur.jpg">

Taking the difference between a pixel and its neighbors detects edges.

\begin{equation}
\begin{bmatrix} 
0 & 0 &  0 & 0 & 0 \\  
0 & 0 &  1 & 0 & 0 \\  
0 & 1 & -4 & 1 & 0 \\  
0 & 0 &  1 & 0 & 0 \\  
0 & 0 &  0 & 0 & 0 \\  
\end{bmatrix}
\end{equation}

<img src="http://docs.gimp.org/en/images/filters/examples/generic-taj-convmatrix-edge-detect.jpg">

You can see other examples in the [GIMP manual](http://docs.gimp.org/en/plug-in-convmatrix.html)


## What are Convolutional Neural Networks?

CNNs are multiple convolution layers paired with a nonlinear activation function. We do not fully connect every layer of a CNN - instead we use _convolutional feature maps_ to create local connections between image pixels. We string together many such convolutional maps, but rather than attempting to pre-select different image enhancements, we treat the filter parameters as wieghts in a neural network and use backpropagation to "train" filter behaviors based on the desired outcomes. By enforcing filter uniformity over an image, we reduce sensitivity to translational invariance (and reduce the number parameters that must be trained). We also compose local patches of low-level features into a high-level representation.

There is another trick CNNs employ, called _pooling_, or subsampling that helps to provide invariance to translation, rotation, and scaling.

Effectively, in sum, the networks learn how to identify edges, then shapes, and then build more complex objects from shapes.

<img src="http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/11/Screen-Shot-2015-11-07-at-7.26.20-AM.png">

(Img source: http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/)

### Sparse Connectivity

CNNs exploit spatially-local correlations by enforcing connectivity patterns between adjacent levels. This means hidden layers are built from subsets of units from lower-level layers, e.g.

<img src="http://deeplearning.net/tutorial/_images/sparse_1D_nn.png">

(Img source: http://deeplearning.net/tutorial/lenet.html)

Stacking many such layers leads to filters that become increasingly global (responsive to larger regions of pixel space) by encoding the features of lower-level layers.

### Shared Weights

In CNNs, each filter is replicated across the entire visual field. Each _feature map_ shares the same parameterization:

<img src="http://deeplearning.net/tutorial/_images/conv_1D_nn.png">

(Img src: http://deeplearning.net/tutorial/lenet.html)

<img src="http://colah.github.io/posts/2014-07-Conv-Nets-Modular/img/Conv2-9x5-Conv2Conv2.png">

(Img src: http://colah.github.io/posts/2014-07-Conv-Nets-Modular/)

We may use the same gradient descent techniques to train shared parameters with only a modest change in the algorithm - the shared weight is simply the sum of the gradients being shared.

## CNN Hyperparameters

### Narrow vs Wide convolution

How do we treat edges when applying a filter? There are two generic choices - "narrow" or "wide" convolutions. In wide convolutions, we zero-pad the image so we may fully apply the filter. In narrow convolutions, we don't do this.

<img src="http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/11/Screen-Shot-2015-11-05-at-9.47.41-AM.png">

(Img source: http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/)

Obviously, wide convolutions are more important when the filter size is large relative to the image. In general, the output size is $n_{out} = (n_{in} + 2 \times n_{padding} - n_{filter}) + 1$.

### Stride Size

_Stride size_ defines how much we want to shift our filter at each step. If the stride size is 1 and the filter is more than one pixel wide, then successive applications will overlap. Larger stride sizes ultimately lead to fewer total applications of the filter.

Stride sizes of 1 are the most common, but larger stride sizes allows the construction of models that look similar to [Recursive Neural Networks](https://en.wikipedia.org/wiki/Recursive_neural_network) (i.e., networks that look like trees). 

### Pooling Layers

CNNs almost always employ pooling layers, usually right after convolutional layers. Pooling layers subsample their inputs (the most common operation is a `max()`):

<img src="http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/11/Screen-Shot-2015-11-05-at-2.18.38-PM.png">

(Img source: http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/, http://cs231n.github.io/convolutional-networks/#pool)

Pooling provides several features - it provides a fixed-size output, it reduces dimensionality (while hopefully preserving the most important information), and it reduces sensitity to shifts and rotations.

Pooling layers act a bit like "zooming out" - they help us figure out if a certain feature was present, but we lose the ability to ask exactly _where_ it was in the image. So we trade-off some location invariance for some loss in our ability to pinpoint location, but this is often a good trade-off.

### Channels

_Channels_ are different "views" of the data. In image recognition, they are classically RGB color channels. Convolutions and pooling may be applied across channels, with different or equal weights.